# Turning Argoverse Data to KITTI 3D Object Detection Format

For this notebook, it only takes the sample dataset. For the real conversion, this will consider all logs in argoverse dataset.

In [1]:
import argoverse
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader

##set root_dir to the correct path to your dataset folder
root_dir =  '../argoverse-tracking/sample/'
argoverse_loader = ArgoverseTrackingLoader(root_dir)
print('Total number of logs:',len(argoverse_loader))
argoverse_loader.print_all()

Total number of logs: 1
#images: 3441
#lidar: 157


In [2]:
argoverse_data = argoverse_loader[0]
print(type(argoverse_data))

<class 'argoverse.data_loading.argoverse_tracking_loader.ArgoverseTrackingLoader'>


### Synchronize stereo pairs with LiDAR point clouds

In [3]:
unique_stereo_left_with_path = argoverse_data.image_list["stereo_front_left"]
unique_stereo_right_with_path = argoverse_data.image_list["stereo_front_right"]

entire_lidar_with_path = argoverse_data._lidar_list[argoverse_data.current_log]

unique_lidar_with_path = {}

for i in range(len(entire_lidar_with_path)):
    if(i%2 == 0):
        # store the pair
        # unique_lidar_with_path.append(entire_lidar_with_path[i])
        unique_lidar_with_path[i] = entire_lidar_with_path[i];
        
if(len(unique_stereo_left_with_path) == len(unique_stereo_right_with_path) == len(unique_lidar_with_path)):
    print("files are synchronized.")
    print("total number of synchornized files for this log =",(len(unique_lidar_with_path)))
else:
    print("there was something wrong during synchronization, please check the files again.")

files are synchronized.
total number of synchornized files for this log = 79


### Annotation Conversion

In [4]:
object_dict = {}

for k in unique_lidar_with_path.keys():
    objects = argoverse_data.get_label_object(k)
    for obj in objects:
        if(obj.label_class in object_dict):
            object_dict[obj.label_class] += 1
        else:
            object_dict[obj.label_class] = 1
    break
            
''' object
        quaternion: np.array,
        translation: np.array,
        length: float,
        width: float,
        height: float,
        occlusion: int,
        label_class: Optional[str] = None,
        track_id: Optional[str] = None,

'''
    
'''   
for i in range(len(unique_lidar_with_path)):
    print(unique_lidar_with_path[i])
    objects = argoverse_data.get_label_object(i)
    for obj in objects:
        if(obj.label_class in object_dict):
            object_dict[obj.label_class] += 1
        else:
            object_dict[obj.label_class] = 1
            
        # 'trasnlation' -> [x,y,z] centroid of an object
        # 'occlusion' -> occlusion value of an object. Argoverse is always 0.
        # 'Bbox'
'''

# total number of classes (objects) throughout the dataset (logs)
print(object_dict)

{'VEHICLE': 49, 'ON_ROAD_OBSTACLE': 7, 'PEDESTRIAN': 2, 'LARGE_VEHICLE': 3, 'MOTORCYCLE': 1}


In [ ]:
objects = argoverse_data.get_label_object(0)
for obj in objects:
    print(obj.label_class)
    print(obj.translation)

In [ ]:
x_mean = temp[:,0].mean()
y_mean = temp[:,1].mean()
z_mean = temp[:,2].mean()

In [ ]:
print(x_mean, y_mean, z_mean)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from argoverse.utils.cv2_plotting_utils import plot_bbox_polygon_cv2
import argoverse.visualization.visualization_utils as viz_util

print(' ------------------------------------------------------------------------')
print(' ------------------------- Ring Cameras ---------------------------------')
print(' ------------------------------------------------------------------------\n')

f,ax = viz_util.make_grid_ring_camera(argoverse_data,0)
plt.show()
print(' ------------------------------------------------------------------------')
print(' ------------------------- Point cloud ---------------------------------')
print(' ------------------------------------------------------------------------\n')

f2 = plt.figure(figsize=(15, 8))
ax2 = f2.add_subplot(111, projection='3d')  
   
viz_util.draw_point_cloud(ax2, 'Lidar scan',argoverse_data,0, xlim3d=[-100,100], ylim3d=[-50,50])
ax2.axis('off')